In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_add_pool
from torch_geometric.datasets import KarateClub, Planetoid, FacebookPagePage
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.data import Data
from sklearn.metrics import normalized_mutual_info_score, f1_score, accuracy_score, adjusted_rand_score
import networkx as nx
from torch_geometric.utils import to_networkx
import numpy as np

class GCNLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNLayer, self).__init__()
        self.conv = GCNConv(in_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv(x, edge_index)
        x = F.relu(x)
        return x

class RNNLayer(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(RNNLayer, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

    def forward(self, x):
        x, (h_n, c_n) = self.rnn(x)
        return x, (h_n, c_n)

class AttentionLayer(nn.Module):
    def __init__(self, in_channels):
        super(AttentionLayer, self).__init__()
        self.attention = nn.Linear(in_channels, 1)

    def forward(self, x):
        weights = F.softmax(self.attention(x), dim=1)
        return torch.sum(weights * x, dim=1)

class HybridModel(nn.Module):
    def __init__(self, gcn_input_dim, gcn_output_dim, rnn_input_dim, rnn_hidden_dim, rnn_num_layers, num_classes):
        super(HybridModel, self).__init__()
        self.gcn = GCNLayer(gcn_input_dim, gcn_output_dim)
        self.rnn = RNNLayer(rnn_input_dim, rnn_hidden_dim, rnn_num_layers)
        self.attention = AttentionLayer(rnn_hidden_dim)
        self.fc = nn.Linear(rnn_hidden_dim, num_classes)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # GCN forward
        x = self.gcn(x, edge_index)
        x = x.unsqueeze(1)

        # RNN forward
        x, (h_n, c_n) = self.rnn(x)

        # Attention Layer
        x = self.attention(x)


        x = self.fc(self.dropout(x))
        return F.log_softmax(x, dim=-1)

def train(model, data, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def test(model, data):
    model.eval()
    logits, accs = model(data), []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

def evaluate(model, data):
    model.eval()
    with torch.no_grad():
        logits = model(data)
        preds = logits.max(1)[1].cpu().numpy()
        labels = data.y.cpu().numpy()


    accuracy = accuracy_score(labels[data.test_mask.cpu()], preds[data.test_mask.cpu()])


    f1 = f1_score(labels[data.test_mask.cpu()], preds[data.test_mask.cpu()], average='macro')


    nmi = normalized_mutual_info_score(labels[data.test_mask.cpu()], preds[data.test_mask.cpu()])


    ari = adjusted_rand_score(labels[data.test_mask.cpu()], preds[data.test_mask.cpu()])


    G = to_networkx(data)
    communities = {i: preds[i] for i in range(len(preds))}
    community_list = [[] for _ in range(data.num_classes)]
    for node, community in communities.items():
        community_list[community].append(node)
    modularity = nx.algorithms.community.modularity(G, community_list)

    return accuracy, f1, nmi, ari, modularity

def load_data(dataset_name, label_rate):
    if dataset_name == 'KarateClub':
        data = KarateClub(transform=NormalizeFeatures())[0]
        data.num_classes = len(set(data.y.tolist()))
    elif dataset_name == 'FacebookPagePage':
        data = FacebookPagePage(root='/tmp/FacebookPagePage', transform=NormalizeFeatures())[0]
        data.num_classes = len(set(data.y.tolist()))
    else:
        data = Planetoid(root=f'/tmp/{dataset_name}', name=dataset_name, transform=NormalizeFeatures())[0]
        data.num_classes = len(set(data.y.tolist()))


    num_nodes = data.y.size(0)
    indices = np.random.permutation(num_nodes)
    train_size = int(num_nodes * label_rate)
    val_size = int(num_nodes * 0.2)
    test_size = num_nodes - train_size - val_size

    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    train_mask[indices[:train_size]] = True
    val_mask[indices[train_size:train_size + val_size]] = True
    test_mask[indices[train_size + val_size:]] = True

    data.train_mask = train_mask
    data.val_mask = val_mask
    data.test_mask = test_mask

    return data

label_rates = [0.1, 0.2, 0.3, 0.5, 1.0]

datasets = ['KarateClub', 'Cora', 'Citeseer', 'FacebookPagePage']
for label_rate in label_rates:
    print(f'Label Rate: {label_rate*100}%')
    for dataset_name in datasets:
        data = load_data(dataset_name, label_rate)


        model = HybridModel(gcn_input_dim=data.num_node_features, gcn_output_dim=32,
                            rnn_input_dim=32, rnn_hidden_dim=64, rnn_num_layers=2, num_classes=data.num_classes)

        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
        criterion = nn.CrossEntropyLoss()


        for epoch in range(200):
            loss = train(model, data, optimizer, criterion)
            if epoch % 10 == 0:
                print(f'Epoch {epoch}, Loss: {loss}')


        accuracy, f1, nmi, ari, modularity = evaluate(model, data)
        print(f'{dataset_name} - Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, NMI: {nmi:.4f}, ARI: {ari:.4f}, Modularity: {modularity:.4f}')
    print('-' * 50)


Label Rate: 10.0%
Epoch 0, Loss: 1.35629141330719
Epoch 10, Loss: 0.800433337688446
Epoch 20, Loss: 0.44208261370658875
Epoch 30, Loss: 0.10981074720621109
Epoch 40, Loss: 0.013080601580440998
Epoch 50, Loss: 0.008519557304680347
Epoch 60, Loss: 0.0007928858976811171
Epoch 70, Loss: 0.00012372522905934602
Epoch 80, Loss: 0.0006349781178869307
Epoch 90, Loss: 0.0002861167013179511
Epoch 100, Loss: 0.00012027110642520711
Epoch 110, Loss: 0.00015538591833319515
Epoch 120, Loss: 0.00012046741176163778
Epoch 130, Loss: 0.0001373826089547947
Epoch 140, Loss: 0.0002617818536236882
Epoch 150, Loss: 0.00011860090307891369
Epoch 160, Loss: 0.00012726329441647977
Epoch 170, Loss: 9.106814832193777e-05
Epoch 180, Loss: 0.00045651933760382235
Epoch 190, Loss: 0.00015402813733089715
KarateClub - Accuracy: 0.4800, F1 Score: 0.3229, NMI: 0.2066, ARI: 0.0961, Modularity: 0.0788
Epoch 0, Loss: 1.9389008283615112
Epoch 10, Loss: 1.8042570352554321
Epoch 20, Loss: 1.5240356922149658
Epoch 30, Loss: 1.1071

C:\Users\AI-BIO\AppData\Roaming\Python\Python311\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Users\AI-BIO\AppData\Roaming\Python\Python311\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


KarateClub - Accuracy: nan, F1 Score: nan, NMI: 1.0000, ARI: 1.0000, Modularity: 0.4156
Epoch 0, Loss: 1.9352468252182007
Epoch 10, Loss: 1.8277878761291504
Epoch 20, Loss: 1.7248204946517944
Epoch 30, Loss: 1.3537386655807495
Epoch 40, Loss: 1.113628625869751
Epoch 50, Loss: 0.8846286535263062
Epoch 60, Loss: 0.6230180859565735
Epoch 70, Loss: 0.42551836371421814
Epoch 80, Loss: 0.27542680501937866
Epoch 90, Loss: 0.16653156280517578
Epoch 100, Loss: 0.11276726424694061
Epoch 110, Loss: 0.08505015820264816
Epoch 120, Loss: 0.06829806417226791
Epoch 130, Loss: 0.051572080701589584
Epoch 140, Loss: 0.04639146104454994
Epoch 150, Loss: 0.03511280193924904
Epoch 160, Loss: 0.03754771500825882
Epoch 170, Loss: 0.028894320130348206
Epoch 180, Loss: 0.03060859814286232
Epoch 190, Loss: 0.028924129903316498


C:\Users\AI-BIO\AppData\Roaming\Python\Python311\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Users\AI-BIO\AppData\Roaming\Python\Python311\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Cora - Accuracy: nan, F1 Score: nan, NMI: 1.0000, ARI: 1.0000, Modularity: 0.6450
Epoch 0, Loss: 1.7981971502304077
Epoch 10, Loss: 1.7533625364303589
Epoch 20, Loss: 1.6559280157089233
Epoch 30, Loss: 1.150996208190918
Epoch 40, Loss: 0.7727739214897156
Epoch 50, Loss: 0.546930730342865
Epoch 60, Loss: 0.39879605174064636
Epoch 70, Loss: 0.29237309098243713
Epoch 80, Loss: 0.22375203669071198
Epoch 90, Loss: 0.17963387072086334
Epoch 100, Loss: 0.14503319561481476
Epoch 110, Loss: 0.1164504885673523
Epoch 120, Loss: 0.10561570525169373
Epoch 130, Loss: 0.09193026274442673
Epoch 140, Loss: 0.0898958295583725
Epoch 150, Loss: 0.08205628395080566
Epoch 160, Loss: 0.07843373715877533
Epoch 170, Loss: 0.07352157682180405
Epoch 180, Loss: 0.0740336999297142
Epoch 190, Loss: 0.06890919804573059


C:\Users\AI-BIO\AppData\Roaming\Python\Python311\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Users\AI-BIO\AppData\Roaming\Python\Python311\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Citeseer - Accuracy: nan, F1 Score: nan, NMI: 1.0000, ARI: 1.0000, Modularity: 0.5635
Epoch 0, Loss: 1.3849437236785889
Epoch 10, Loss: 1.351935863494873
Epoch 20, Loss: 1.2983276844024658
Epoch 30, Loss: 0.9937758445739746
Epoch 40, Loss: 0.8047879934310913
Epoch 50, Loss: 0.6527915000915527
Epoch 60, Loss: 0.5521327257156372
Epoch 70, Loss: 0.4916626811027527
Epoch 80, Loss: 0.4633161127567291
Epoch 90, Loss: 0.43749964237213135
Epoch 100, Loss: 0.42642003297805786
Epoch 110, Loss: 0.41782695055007935
Epoch 120, Loss: 0.41025811433792114
Epoch 130, Loss: 0.40341079235076904
Epoch 140, Loss: 0.407344788312912
Epoch 150, Loss: 0.3978734314441681
Epoch 160, Loss: 0.39045581221580505
Epoch 170, Loss: 0.3857491910457611
Epoch 180, Loss: 0.39240550994873047
Epoch 190, Loss: 0.3778238296508789


C:\Users\AI-BIO\AppData\Roaming\Python\Python311\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Users\AI-BIO\AppData\Roaming\Python\Python311\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


FacebookPagePage - Accuracy: nan, F1 Score: nan, NMI: 1.0000, ARI: 1.0000, Modularity: 0.5135
--------------------------------------------------


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_add_pool
from torch_geometric.datasets import KarateClub, Planetoid, FacebookPagePage
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.data import Data
from sklearn.metrics import normalized_mutual_info_score, f1_score, accuracy_score, adjusted_rand_score
import networkx as nx
from torch_geometric.utils import to_networkx
import numpy as np

class GCNLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNLayer, self).__init__()
        self.conv = GCNConv(in_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv(x, edge_index)
        x = F.relu(x)
        return x

class RNNLayer(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(RNNLayer, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

    def forward(self, x):
        x, (h_n, c_n) = self.rnn(x)
        return x, (h_n, c_n)

class AttentionLayer(nn.Module):
    def __init__(self, in_channels):
        super(AttentionLayer, self).__init__()
        self.attention = nn.Linear(in_channels, 1)

    def forward(self, x):
        weights = F.softmax(self.attention(x), dim=1)
        return torch.sum(weights * x, dim=1)

class HybridModel(nn.Module):
    def __init__(self, gcn_input_dim, gcn_output_dim, rnn_input_dim, rnn_hidden_dim, rnn_num_layers, num_classes):
        super(HybridModel, self).__init__()
        self.gcn = GCNLayer(gcn_input_dim, gcn_output_dim)
        self.rnn = RNNLayer(rnn_input_dim, rnn_hidden_dim, rnn_num_layers)
        self.attention = AttentionLayer(rnn_hidden_dim)
        self.fc = nn.Linear(rnn_hidden_dim, num_classes)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # GCN forward
        x = self.gcn(x, edge_index)
        x = x.unsqueeze(1)

        # RNN forward
        x, (h_n, c_n) = self.rnn(x)

        # Attention Layer
        x = self.attention(x)


        x = self.fc(self.dropout(x))
        return F.log_softmax(x, dim=-1)

def train(model, data, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def test(model, data):
    model.eval()
    logits, accs = model(data), []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

def evaluate(model, data):
    model.eval()
    with torch.no_grad():
        logits = model(data)
        preds = logits.max(1)[1].cpu().numpy()
        labels = data.y.cpu().numpy()


    accuracy = accuracy_score(labels[data.test_mask.cpu()], preds[data.test_mask.cpu()])


    f1 = f1_score(labels[data.test_mask.cpu()], preds[data.test_mask.cpu()], average='macro')


    nmi = normalized_mutual_info_score(labels[data.test_mask.cpu()], preds[data.test_mask.cpu()])


    ari = adjusted_rand_score(labels[data.test_mask.cpu()], preds[data.test_mask.cpu()])


    G = to_networkx(data)
    communities = {i: preds[i] for i in range(len(preds))}
    community_list = [[] for _ in range(data.num_classes)]
    for node, community in communities.items():
        community_list[community].append(node)
    modularity = nx.algorithms.community.modularity(G, community_list)

    return accuracy, f1, nmi, ari, modularity

def load_data(dataset_name, label_rate):
    if dataset_name == 'KarateClub':
        data = KarateClub(transform=NormalizeFeatures())[0]
        data.num_classes = len(set(data.y.tolist()))
    elif dataset_name == 'FacebookPagePage':
        data = FacebookPagePage(root='/tmp/FacebookPagePage', transform=NormalizeFeatures())[0]
        data.num_classes = len(set(data.y.tolist()))
    else:
        data = Planetoid(root=f'/tmp/{dataset_name}', name=dataset_name, transform=NormalizeFeatures())[0]
        data.num_classes = len(set(data.y.tolist()))


    num_nodes = data.y.size(0)
    indices = np.random.permutation(num_nodes)
    train_size = int(num_nodes * label_rate)
    val_size = int(num_nodes * 0.2)
    test_size = num_nodes - train_size - val_size

    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    train_mask[indices[:train_size]] = True
    val_mask[indices[train_size:train_size + val_size]] = True
    test_mask[indices[train_size + val_size:]] = True

    data.train_mask = train_mask
    data.val_mask = val_mask
    data.test_mask = test_mask

    return data

label_rates = [0.5]

datasets = ['KarateClub', 'Cora', 'Citeseer', 'FacebookPagePage']
for label_rate in label_rates:
    print(f'Label Rate: {label_rate*100}%')
    for dataset_name in datasets:
        data = load_data(dataset_name, label_rate)


        model = HybridModel(gcn_input_dim=data.num_node_features, gcn_output_dim=32,
                            rnn_input_dim=32, rnn_hidden_dim=64, rnn_num_layers=2, num_classes=data.num_classes)

        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
        criterion = nn.CrossEntropyLoss()


        for epoch in range(180):
            loss = train(model, data, optimizer, criterion)
            if epoch % 10 == 0:
                print(f'Epoch {epoch}, Loss: {loss}')


        accuracy, f1, nmi, ari, modularity = evaluate(model, data)
        print(f'{dataset_name} - Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, NMI: {nmi:.4f}, ARI: {ari:.4f}, Modularity: {modularity:.4f}')
    print('-' * 50)


Label Rate: 50.0%
Epoch 0, Loss: 1.3711448907852173
Epoch 10, Loss: 1.1305335760116577
Epoch 20, Loss: 0.45783746242523193
Epoch 30, Loss: 0.08745983242988586
Epoch 40, Loss: 0.01846112310886383
Epoch 50, Loss: 0.0027366559952497482
Epoch 60, Loss: 0.0006068044458515942
Epoch 70, Loss: 0.0019606370478868484
Epoch 80, Loss: 0.0006605358212254941
Epoch 90, Loss: 0.0007238892139866948
Epoch 100, Loss: 0.0005809016874991357
Epoch 110, Loss: 8.955871453508735e-05
Epoch 120, Loss: 0.0002976180985569954
Epoch 130, Loss: 0.00014285063662100583
Epoch 140, Loss: 0.00021204205404501408
Epoch 150, Loss: 0.0005727570387534797
Epoch 160, Loss: 0.0001968852011486888
Epoch 170, Loss: 0.00038805505027994514
KarateClub - Accuracy: 0.7273, F1 Score: 0.6423, NMI: 0.5859, ARI: 0.3271, Modularity: 0.3574
Epoch 0, Loss: 1.9673899412155151
Epoch 10, Loss: 1.8166580200195312
Epoch 20, Loss: 1.6866652965545654
Epoch 30, Loss: 1.3854844570159912
Epoch 40, Loss: 1.1671066284179688
Epoch 50, Loss: 0.98047631978988

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_add_pool
from torch_geometric.datasets import KarateClub, Planetoid, FacebookPagePage
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.data import Data
from sklearn.metrics import normalized_mutual_info_score, f1_score, accuracy_score, adjusted_rand_score
import networkx as nx
from torch_geometric.utils import to_networkx
import numpy as np

class GCNLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNLayer, self).__init__()
        self.conv = GCNConv(in_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv(x, edge_index)
        x = F.relu(x)
        return x

class RNNLayer(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(RNNLayer, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

    def forward(self, x):
        x, (h_n, c_n) = self.rnn(x)
        return x, (h_n, c_n)

class AttentionLayer(nn.Module):
    def __init__(self, in_channels):
        super(AttentionLayer, self).__init__()
        self.attention = nn.Linear(in_channels, 1)

    def forward(self, x):
        weights = F.softmax(self.attention(x), dim=1)
        return torch.sum(weights * x, dim=1)

class HybridModel(nn.Module):
    def __init__(self, gcn_input_dim, gcn_output_dim, rnn_input_dim, rnn_hidden_dim, rnn_num_layers, num_classes):
        super(HybridModel, self).__init__()
        self.gcn = GCNLayer(gcn_input_dim, gcn_output_dim)
        self.rnn = RNNLayer(rnn_input_dim, rnn_hidden_dim, rnn_num_layers)
        self.attention = AttentionLayer(rnn_hidden_dim)
        self.fc = nn.Linear(rnn_hidden_dim, num_classes)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.gcn(x, edge_index)
        x = x.unsqueeze(1)
        x, (h_n, c_n) = self.rnn(x)
        x = self.attention(x)
        x = self.fc(self.dropout(x))
        return F.log_softmax(x, dim=-1)

def train(model, data, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def test(model, data):
    model.eval()
    logits, accs = model(data), []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

def evaluate(model, data):
    model.eval()
    with torch.no_grad():
        logits = model(data)
        preds = logits.max(1)[1].cpu().numpy()
        labels = data.y.cpu().numpy()

    accuracy = accuracy_score(labels[data.test_mask.cpu()], preds[data.test_mask.cpu()]) if torch.count_nonzero(data.test_mask) > 0 else float('nan')
    f1 = f1_score(labels[data.test_mask.cpu()], preds[data.test_mask.cpu()], average='macro') if torch.count_nonzero(data.test_mask) > 0 else float('nan')
    nmi = normalized_mutual_info_score(labels[data.test_mask.cpu()], preds[data.test_mask.cpu()]) if torch.count_nonzero(data.test_mask) > 0 else float('nan')
    ari = adjusted_rand_score(labels[data.test_mask.cpu()], preds[data.test_mask.cpu()]) if torch.count_nonzero(data.test_mask) > 0 else float('nan')

    if torch.count_nonzero(data.test_mask) > 0:
        G = to_networkx(data)
        communities = {i: preds[i] for i in range(len(preds))}
        community_list = [[] for _ in range(data.num_classes)]
        for node, community in communities.items():
            community_list[community].append(node)
        modularity = nx.algorithms.community.modularity(G, community_list)
    else:
        modularity = float('nan')

    return accuracy, f1, nmi, ari, modularity

def load_data(dataset_name, label_rate):
    if dataset_name == 'KarateClub':
        data = KarateClub(transform=NormalizeFeatures())[0]
        data.num_classes = len(set(data.y.tolist()))
    elif dataset_name == 'FacebookPagePage':
        data = FacebookPagePage(root='/tmp/FacebookPagePage', transform=NormalizeFeatures())[0]
        data.num_classes = len(set(data.y.tolist()))
    else:
        data = Planetoid(root=f'/tmp/{dataset_name}', name=dataset_name, transform=NormalizeFeatures())[0]
        data.num_classes = len(set(data.y.tolist()))

    num_nodes = data.y.size(0)
    indices = np.random.permutation(num_nodes)
    train_size = int(num_nodes * label_rate * 0.8)
    val_size = int(num_nodes * 0.1)
    test_size = num_nodes - train_size - val_size

    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    train_mask[indices[:train_size]] = True
    val_mask[indices[train_size:train_size + val_size]] = True
    test_mask[indices[train_size + val_size:]] = True

    data.train_mask = train_mask
    data.val_mask = val_mask
    data.test_mask = test_mask

    return data

label_rates = [1.0]
datasets = ['KarateClub', 'Cora', 'Citeseer', 'FacebookPagePage']

for label_rate in label_rates:
    print(f'Label Rate: {label_rate*100}%')
    for dataset_name in datasets:
        data = load_data(dataset_name, label_rate)

        model = HybridModel(gcn_input_dim=data.num_node_features, gcn_output_dim=32,
                            rnn_input_dim=32, rnn_hidden_dim=64, rnn_num_layers=2, num_classes=data.num_classes)

        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
        criterion = nn.CrossEntropyLoss()

        for epoch in range(200):
            loss = train(model, data, optimizer, criterion)
            if epoch % 10 == 0:
                print(f'Epoch {epoch}, Loss: {loss}')

        accuracy, f1, nmi, ari, modularity = evaluate(model, data)
        print(f'{dataset_name} - Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, NMI: {nmi:.4f}, ARI: {ari:.4f}, Modularity: {modularity:.4f}')
    print('-' * 50)


Label Rate: 100.0%
Epoch 0, Loss: 1.3896249532699585
Epoch 10, Loss: 1.1538077592849731
Epoch 20, Loss: 0.5244362354278564
Epoch 30, Loss: 0.15389834344387054
Epoch 40, Loss: 0.025691548362374306
Epoch 50, Loss: 0.002779630245640874
Epoch 60, Loss: 0.001739880652166903
Epoch 70, Loss: 0.0002869184536393732
Epoch 80, Loss: 0.0023265762720257044
Epoch 90, Loss: 0.0005375660257413983
Epoch 100, Loss: 0.0009038196294568479
Epoch 110, Loss: 0.00044713864917866886
Epoch 120, Loss: 0.0002655461139511317
Epoch 130, Loss: 0.0006343198474496603
Epoch 140, Loss: 0.0003869888896588236
Epoch 150, Loss: 0.0003263674443587661
Epoch 160, Loss: 0.00016747225890867412
Epoch 170, Loss: 0.00014289995306171477
Epoch 180, Loss: 0.000239127068198286
Epoch 190, Loss: 0.0005173737881705165
KarateClub - Accuracy: 0.7500, F1 Score: 0.6000, NMI: 0.7020, ARI: 0.3333, Modularity: 0.3638
Epoch 0, Loss: 1.9273933172225952
Epoch 10, Loss: 1.8261311054229736
Epoch 20, Loss: 1.675776720046997
Epoch 30, Loss: 1.303193330

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_add_pool
from torch_geometric.datasets import KarateClub, Planetoid, FacebookPagePage
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.data import Data
from sklearn.metrics import normalized_mutual_info_score, f1_score, accuracy_score, adjusted_rand_score
import networkx as nx
from torch_geometric.utils import to_networkx
import numpy as np

class GCNLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNLayer, self).__init__()
        self.conv = GCNConv(in_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv(x, edge_index)
        x = F.relu(x)
        return x

class RNNLayer(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(RNNLayer, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

    def forward(self, x):
        x, (h_n, c_n) = self.rnn(x)
        return x, (h_n, c_n)

class AttentionLayer(nn.Module):
    def __init__(self, in_channels):
        super(AttentionLayer, self).__init__()
        self.attention = nn.Linear(in_channels, 1)

    def forward(self, x):
        weights = F.softmax(self.attention(x), dim=1)
        return torch.sum(weights * x, dim=1)

class HybridModel(nn.Module):
    def __init__(self, gcn_input_dim, gcn_output_dim, rnn_input_dim, rnn_hidden_dim, rnn_num_layers, num_classes):
        super(HybridModel, self).__init__()
        self.gcn = GCNLayer(gcn_input_dim, gcn_output_dim)
        self.rnn = RNNLayer(rnn_input_dim, rnn_hidden_dim, rnn_num_layers)
        self.attention = AttentionLayer(rnn_hidden_dim)
        self.fc = nn.Linear(rnn_hidden_dim, num_classes)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.gcn(x, edge_index)
        x = x.unsqueeze(1)
        x, (h_n, c_n) = self.rnn(x)
        x = self.attention(x)
        x = self.fc(self.dropout(x))
        return F.log_softmax(x, dim=-1)

def train(model, data, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def test(model, data):
    model.eval()
    logits, accs = model(data), []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

def evaluate(model, data):
    model.eval()
    with torch.no_grad():
        logits = model(data)
        preds = logits.max(1)[1].cpu().numpy()
        labels = data.y.cpu().numpy()

    accuracy = accuracy_score(labels[data.test_mask.cpu()], preds[data.test_mask.cpu()]) if torch.count_nonzero(data.test_mask) > 0 else float('nan')
    f1 = f1_score(labels[data.test_mask.cpu()], preds[data.test_mask.cpu()], average='macro') if torch.count_nonzero(data.test_mask) > 0 else float('nan')
    nmi = normalized_mutual_info_score(labels[data.test_mask.cpu()], preds[data.test_mask.cpu()]) if torch.count_nonzero(data.test_mask) > 0 else float('nan')
    ari = adjusted_rand_score(labels[data.test_mask.cpu()], preds[data.test_mask.cpu()]) if torch.count_nonzero(data.test_mask) > 0 else float('nan')

    if torch.count_nonzero(data.test_mask) > 0:
        G = to_networkx(data)
        communities = {i: preds[i] for i in range(len(preds))}
        community_list = [[] for _ in range(data.num_classes)]
        for node, community in communities.items():
            community_list[community].append(node)
        modularity = nx.algorithms.community.modularity(G, community_list)
    else:
        modularity = float('nan')

    return accuracy, f1, nmi, ari, modularity

def load_data(dataset_name, label_rate):
    if dataset_name == 'KarateClub':
        data = KarateClub(transform=NormalizeFeatures())[0]
        data.num_classes = len(set(data.y.tolist()))
    elif dataset_name == 'FacebookPagePage':
        data = FacebookPagePage(root='/tmp/FacebookPagePage', transform=NormalizeFeatures())[0]
        data.num_classes = len(set(data.y.tolist()))
    else:
        data = Planetoid(root=f'/tmp/{dataset_name}', name=dataset_name, transform=NormalizeFeatures())[0]
        data.num_classes = len(set(data.y.tolist()))

    num_nodes = data.y.size(0)
    indices = np.random.permutation(num_nodes)
    train_size = int(num_nodes * label_rate * 0.8)
    val_size = int(num_nodes * 0.1)
    test_size = num_nodes - train_size - val_size

    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    train_mask[indices[:train_size]] = True
    val_mask[indices[train_size:train_size + val_size]] = True
    test_mask[indices[train_size + val_size:]] = True

    data.train_mask = train_mask
    data.val_mask = val_mask
    data.test_mask = test_mask

    return data

label_rates = [1.0]
datasets = ['KarateClub', 'Cora', 'Citeseer', 'FacebookPagePage']

for label_rate in label_rates:
    print(f'Label Rate: {label_rate*100}%')
    for dataset_name in datasets:
        data = load_data(dataset_name, label_rate)

        model = HybridModel(gcn_input_dim=data.num_node_features, gcn_output_dim=32,
                            rnn_input_dim=32, rnn_hidden_dim=64, rnn_num_layers=2, num_classes=data.num_classes)

        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
        criterion = nn.CrossEntropyLoss()

        for epoch in range(310):
            loss = train(model, data, optimizer, criterion)
            if epoch % 10 == 0:
                print(f'Epoch {epoch}, Loss: {loss}')

        accuracy, f1, nmi, ari, modularity = evaluate(model, data)
        print(f'{dataset_name} - Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, NMI: {nmi:.4f}, ARI: {ari:.4f}, Modularity: {modularity:.4f}')
    print('-' * 50)


Label Rate: 100.0%
Epoch 0, Loss: 1.3688477277755737
Epoch 10, Loss: 1.150694489479065
Epoch 20, Loss: 0.5094664096832275
Epoch 30, Loss: 0.2290314882993698
Epoch 40, Loss: 0.044556599110364914
Epoch 50, Loss: 0.009075943380594254
Epoch 60, Loss: 0.0014017247594892979
Epoch 70, Loss: 0.0005230701644904912
Epoch 80, Loss: 0.0003734665224328637
Epoch 90, Loss: 0.0007931272266432643
Epoch 100, Loss: 0.0006164037040434778
Epoch 110, Loss: 0.00012113459524698555
Epoch 120, Loss: 0.00046678713988512754
Epoch 130, Loss: 0.0005480805994011462
Epoch 140, Loss: 0.00029284681659191847
Epoch 150, Loss: 0.00027710673748515546
Epoch 160, Loss: 0.0004492539737839252
Epoch 170, Loss: 0.00030655748560093343
Epoch 180, Loss: 0.0003795860684476793
Epoch 190, Loss: 0.00014023740368429571
Epoch 200, Loss: 0.00024865646264515817
Epoch 210, Loss: 0.0003798681136686355
Epoch 220, Loss: 0.00016028116806410253
Epoch 230, Loss: 0.00020101285190321505
Epoch 240, Loss: 0.0005142365698702633
Epoch 250, Loss: 0.0003

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import KarateClub, Planetoid, FacebookPagePage
from torch_geometric.transforms import NormalizeFeatures
from sklearn.metrics import normalized_mutual_info_score, f1_score, accuracy_score, adjusted_rand_score
import networkx as nx
from torch_geometric.utils import to_networkx
import numpy as np

class GCNLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNLayer, self).__init__()
        self.conv = GCNConv(in_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv(x, edge_index)
        x = F.relu(x)
        return x

class RNNLayer(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(RNNLayer, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

    def forward(self, x):
        x, (h_n, c_n) = self.rnn(x)
        return x, (h_n, c_n)

class AttentionLayer(nn.Module):
    def __init__(self, in_channels):
        super(AttentionLayer, self).__init__()
        self.attention = nn.Linear(in_channels, 1)

    def forward(self, x):
        weights = F.softmax(self.attention(x), dim=1)
        return torch.sum(weights * x, dim=1)

class HybridModel(nn.Module):
    def __init__(self, gcn_input_dim, gcn_output_dim, rnn_input_dim, rnn_hidden_dim, rnn_num_layers, num_classes):
        super(HybridModel, self).__init__()
        self.gcn = GCNLayer(gcn_input_dim, gcn_output_dim)
        self.rnn = RNNLayer(rnn_input_dim, rnn_hidden_dim, rnn_num_layers)
        self.attention = AttentionLayer(rnn_hidden_dim)
        self.fc = nn.Linear(rnn_hidden_dim, num_classes)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.gcn(x, edge_index)
        x = x.unsqueeze(1)
        x, (h_n, c_n) = self.rnn(x)
        x = self.attention(x)
        x = self.fc(self.dropout(x))
        return F.log_softmax(x, dim=-1)

def train(model, data, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(model, data):
    model.eval()
    with torch.no_grad():
        logits = model(data)
        preds = logits.max(1)[1].cpu().numpy()
        labels = data.y.cpu().numpy()

        accuracy = accuracy_score(labels[data.test_mask.cpu()], preds[data.test_mask.cpu()]) if torch.count_nonzero(data.test_mask) > 0 else float('nan')
        f1 = f1_score(labels[data.test_mask.cpu()], preds[data.test_mask.cpu()], average='macro') if torch.count_nonzero(data.test_mask) > 0 else float('nan')
        nmi = normalized_mutual_info_score(labels[data.test_mask.cpu()], preds[data.test_mask.cpu()]) if torch.count_nonzero(data.test_mask) > 0 else float('nan')
        ari = adjusted_rand_score(labels[data.test_mask.cpu()], preds[data.test_mask.cpu()]) if torch.count_nonzero(data.test_mask) > 0 else float('nan')

        if torch.count_nonzero(data.test_mask) > 0:
            G = to_networkx(data)
            communities = {i: preds[i] for i in range(len(preds))}
            community_list = [[] for _ in range(data.num_classes)]
            for node, community in communities.items():
                community_list[community].append(node)
            modularity = nx.algorithms.community.modularity(G, community_list)
        else:
            modularity = float('nan')

        return accuracy, f1, nmi, ari, modularity

def load_data(dataset_name, label_rate):
    if dataset_name == 'KarateClub':
        data = KarateClub(transform=NormalizeFeatures())[0]
        data.num_classes = len(set(data.y.tolist()))
    elif dataset_name == 'FacebookPagePage':
        data = FacebookPagePage(root='/tmp/FacebookPagePage', transform=NormalizeFeatures())[0]
        data.num_classes = len(set(data.y.tolist()))
    else:
        data = Planetoid(root=f'/tmp/{dataset_name}', name=dataset_name, transform=NormalizeFeatures())[0]
        data.num_classes = len(set(data.y.tolist()))

    num_nodes = data.y.size(0)
    indices = np.random.permutation(num_nodes)
    train_size = int(num_nodes * label_rate * 0.8)
    val_size = int(num_nodes * 0.1)
    test_size = num_nodes - train_size - val_size

    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    train_mask[indices[:train_size]] = True
    val_mask[indices[train_size:train_size + val_size]] = True
    test_mask[indices[train_size + val_size:]] = True

    data.train_mask = train_mask
    data.val_mask = val_mask
    data.test_mask = test_mask

    return data

label_rates = [1.0]
datasets = ['KarateClub']

for label_rate in label_rates:
    print(f'Label Rate: {label_rate*100}%')
    for dataset_name in datasets:
        data = load_data(dataset_name, label_rate)

        model = HybridModel(gcn_input_dim=data.num_node_features, gcn_output_dim=32,
                            rnn_input_dim=32, rnn_hidden_dim=64, rnn_num_layers=2, num_classes=data.num_classes)

        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
        criterion = nn.CrossEntropyLoss()

        for epoch in range(160):
            loss = train(model, data, optimizer, criterion)
            if epoch % 10 == 0:
                print(f'Epoch {epoch}, Loss: {loss}')

        accuracy, f1, nmi, ari, modularity = evaluate(model, data)
        print(f'{dataset_name} - Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, NMI: {nmi:.4f}, ARI: {ari:.4f}, Modularity: {modularity:.4f}')
    print('-' * 50)


Label Rate: 100.0%
Epoch 0, Loss: 1.3783901929855347
Epoch 10, Loss: 1.2081594467163086
Epoch 20, Loss: 0.6487535834312439
Epoch 30, Loss: 0.12527935206890106
Epoch 40, Loss: 0.0029193072114139795
Epoch 50, Loss: 0.002836272120475769
Epoch 60, Loss: 0.0005893099587410688
Epoch 70, Loss: 0.0006468232022598386
Epoch 80, Loss: 0.0005300342454575002
Epoch 90, Loss: 0.0005520393024198711
Epoch 100, Loss: 0.00020218607096467167
Epoch 110, Loss: 0.00019377554417587817
Epoch 120, Loss: 0.0001147677467088215
Epoch 130, Loss: 0.0002388063003309071
Epoch 140, Loss: 6.855453102616593e-05
Epoch 150, Loss: 0.0002886205620598048
KarateClub - Accuracy: 0.7500, F1 Score: 0.6000, NMI: 0.7020, ARI: 0.3333, Modularity: 0.3410
--------------------------------------------------
